# Разметка данных

# Пакеты

In [1]:
import pandas as pd
from enum import Enum
from datasets import Dataset, DatasetDict

In [2]:
df = pd.read_csv("places_address_elements.csv")
df = df.fillna('')

In [3]:
df

,region_name,region_type_name,region_type_name_short,area_name,area_type_name,area_type_name_short,city_name,city_type_name,city_type_name_short,territory_name,territory_type_name,territory_type_name_short,street_name,street_type_name,street_type_name_short
0,Адыгея,Республика,Респ,,,,,,,,,,,,
1,Башкортостан,Республика,Респ,,,,,,,,,,,,
2,Алтай,Республика,Респ,,,,,,,,,,,,
3,Кабардино-Балкарская,Республика,Респ,,,,,,,,,,,,
4,Калмыкия,Республика,Респ,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519314,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Малиновая,Улица,ул.
1519315,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Персиковая,Улица,ул.
1519316,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Яблоневая,Улица,ул.
1519317,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Садовая,Улица,ул.


# Словарь Entities

In [4]:

class ElementType(Enum):
    OTHER = 0
    REGION=1
    REGION_TYPE=2
    AREA=3
    AREA_TYPE=4
    TERRITORY=5
    TERRITORY_TYPE=6
    CITY=7
    CITY_TYPE=8
    STREET=9
    STREET_TYPE=10


# Представление адреса в виде слов-токенов и тегов NER

In [5]:
str ='Алтайский Край, Залесовский Район, Залесово Село, автодорога Мартыново-Тогул-Залесово Территория, 151-й Километр'
tokens = str.split(' ')
tokens

['Алтайский',
 'Край,',
 'Залесовский',
 'Район,',
 'Залесово',
 'Село,',
 'автодорога',
 'Мартыново-Тогул-Залесово',
 'Территория,',
 '151-й',
 'Километр']

In [6]:
class TypedWord:
    word = ''
    type = ElementType.OTHER
    def __init__(self, word:str, type:ElementType):
        self.word = word
        self.type = type
    
    def __repr__(self):
        return self.word + ":" + self.type.name

### Функция конвертации набора адресо-образующих элементов в токены и NER-теги

In [7]:
def tokens_with_tag(input , elementType :ElementType  ) -> list:
    words = list(filter(('').__ne__, input.split(' ')))
    res = list()
    for word in words:
        res.append(TypedWord(word, elementType))
    return res


def convert_address_elements(elements: dict):
    return (tokens_with_tag(elements['region_name'], ElementType.REGION) +
            tokens_with_tag(elements['region_type_name'], ElementType.REGION_TYPE) +
            tokens_with_tag(elements['area_name'], ElementType.AREA) +
            tokens_with_tag(elements['area_type_name'], ElementType.AREA_TYPE) +
            tokens_with_tag(elements['city_name'], ElementType.CITY) +
            tokens_with_tag(elements['city_type_name'], ElementType.CITY_TYPE) +
            tokens_with_tag(elements['territory_name'], ElementType.TERRITORY) +
            tokens_with_tag(elements['territory_type_name'], ElementType.TERRITORY_TYPE) +
            tokens_with_tag(elements['street_name'], ElementType.STREET) +
            tokens_with_tag(elements['street_type_name'], ElementType.STREET_TYPE)
            )

# elements = df.sample().to_dict('records')
# convert_address_elements(elements[0])

[Краснодарский:REGION,
 Край:REGION_TYPE,
 Сочи:CITY,
 Город:CITY_TYPE,
 Вишневый:TERRITORY,
 Садовое:TERRITORY_TYPE,
 товарищество:TERRITORY_TYPE]

In [14]:
convert_address_elements(df.sample().to_dict('records')[0])

[Иркутская:REGION,
 Область:REGION_TYPE,
 Нижнеудинский:AREA,
 Район:AREA_TYPE,
 Костино:CITY,
 Поселок:CITY_TYPE,
 Сосновая:STREET,
 Улица:STREET_TYPE]

## Appply to dataframe

In [11]:
df['typed_words'] = df.apply(convert_address_elements, axis=1)

In [22]:
typed_words = pd.DataFrame(df['typed_words'])

In [23]:
typed_words

,typed_words
0,"[Адыгея:REGION, Республика:REGION_TYPE]"
1,"[Башкортостан:REGION, Республика:REGION_TYPE]"
2,"[Алтай:REGION, Республика:REGION_TYPE]"
3,"[Кабардино-Балкарская:REGION, Республика:REGIO..."
4,"[Калмыкия:REGION, Республика:REGION_TYPE]"
...,...
1519314,"[Краснодарский:REGION, Край:REGION_TYPE, Тихор..."
1519315,"[Краснодарский:REGION, Край:REGION_TYPE, Тихор..."
1519316,"[Краснодарский:REGION, Край:REGION_TYPE, Тихор..."
1519317,"[Краснодарский:REGION, Край:REGION_TYPE, Тихор..."


## В формате Datasets

In [24]:
dataset = Dataset.from_pandas(typed_words)

ArrowInvalid: ('Could not convert Адыгея:REGION with type TypedWord: did not recognize Python value type when inferring an Arrow data type', 'Conversion failed for column typed_words with type object')

In [74]:
train_testvalid = dataset.train_test_split(test_size=0.33, seed=42)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'validation': test_valid['train'],
    'test': test_valid['test']})

In [75]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 410596
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 101117
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 101117
    })
})

In [76]:
dataset.save_to_disk('address_elements')

Saving the dataset (0/1 shards):   0%|          | 0/410596 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/101117 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/101117 [00:00<?, ? examples/s]